## Imports

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import pandas as pd
import math 
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
import pickle
import warnings
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    warnings.warn('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))

## Selección de sets

In [ ]:
#leemos los datos (nibaldo)
train_set=pd.read_pickle('/gdrive/My Drive/Colab Notebooks/CNNAlerce/train_set.pkl', compression='infer')
val_set=pd.read_pickle('/gdrive/My Drive/Colab Notebooks/CNNAlerce/val_set.pkl', compression='infer')
test_set=pd.read_pickle('/gdrive/My Drive/Colab Notebooks/CNNAlerce/test_set.pkl', compression='infer')


'''
#para diego:
train_set=pd.read_pickle('/gdrive/My Drive/CNNAlerce/train_set.pkl', compression='infer')
val_set=pd.read_pickle('/gdrive/My Drive/CNNAlerce/val_set.pkl', compression='infer')
test_set=pd.read_pickle('/gdrive/My Drive/CNNAlerce/test_set.pkl', compression='infer')
'''

#Construcción de sets:
x_train = np.array(train_set['im_21']) #['images'] para 63 pixeles
x_val = np.array(val_set['im_21']) #['images'] para 63 pixeles
x_test = np.array(test_set['im_21']) #['images'] para 63 pixeles
y_train = np.array(train_set['labels'])
y_val = np.array(val_set['labels'])
y_test = np.array(test_set['labels'])
del train_set
del val_set
del test_set

##Modelos sin Invarianza Rotacional


### Modelo preliminar 




In [ ]:
#Clasificador preliminar
model = tf.keras.Sequential()

#Parte convolucional
model.add(tf.keras.layers.Conv2D(16, kernel_size=(3, 3), input_shape=(21, 21, 3), activation='relu'))#, padding='same'))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))#, padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))#, padding='same'))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))#, padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#Fully-connected
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer, loss, metrics=['accuracy'])

In [ ]:
#Entrenamiento:
summary = model.fit(x=x_train, y=y_train, batch_size=1460, epochs=20, verbose=1, validation_data=(x_val, y_val), shuffle=True, callbacks=[tf.keras.callbacks.TensorBoard('./logs/preliminar', update_freq=100)])

In [ ]:
#Testing
model.evaluate(x=x_test, y=y_test, batch_size=32, verbose=1)

### Modelo paper sin rotaciones

In [ ]:
#Clasificador paper
model = tf.keras.Sequential()

#model.add(tf.keras.layers.ZeroPadding2D(padding=((3,3),(3,3)), data_format=None))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(4, 4), input_shape=(21, 21, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer, loss, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#Entrenamiento:
nombre_ensayo = './logs/sroverfitting_3'
summary = model.fit(x=x_train, y=y_train, batch_size=1460, epochs=19, verbose=1, validation_data=(x_val, y_val), shuffle=True, callbacks=[tf.keras.callbacks.TensorBoard(nombre_ensayo, update_freq=100)])

In [ ]:
#Testing
model.evaluate(x=x_test, y=y_test, batch_size=32, verbose=1)

## Modelo con Invarianza Rotacional

### Construcción Red

In [ ]:
img_inputs = tf.keras.Input(shape=(21, 21, 3))
rot_90 = tf.keras.layers.Lambda(lambda x: tf.image.rot90(x, k=1))(img_inputs)
rot_180 = tf.keras.layers.Lambda(lambda x: tf.image.rot90(x, k=2))(img_inputs)
rot_270 = tf.keras.layers.Lambda(lambda x: tf.image.rot90(x, k=3))(img_inputs)

#RED COMPARTIDA
conv_1 = tf.keras.layers.Conv2D(32, kernel_size=(4, 4), padding='same', activation='relu')#(img_inputs)
conv_2 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')#(conv_1)
max_pool_1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))#(conv_2)
conv_3 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')#(max_pool_1)
conv_4 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')#(conv_3)
conv_5 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')#(conv_4)
max_pool_2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))#(conv_5)

#
out1 = conv_1(img_inputs)
out1 = conv_2(out1)
out1 = max_pool_1(out1)
out1 = conv_3(out1)
out1 = conv_4(out1)
out1 = conv_5(out1)
out1 = max_pool_2(out1)
#
out2 = conv_1(rot_90)
out2 = conv_2(out2)
out2 = max_pool_1(out2)
out2 = conv_3(out2)
out2 = conv_4(out2)
out2 = conv_5(out2)
out2 = max_pool_2(out2)
#
out3 = conv_1(rot_180)
out3 = conv_2(out3)
out3 = max_pool_1(out3)
out3 = conv_3(out3)
out3 = conv_4(out3)
out3 = conv_5(out3)
out3 = max_pool_2(out3)
#
out4 = conv_1(rot_270)
out4 = conv_2(out4)
out4 = max_pool_1(out4)
out4 = conv_3(out4)
out4 = conv_4(out4)
out4 = conv_5(out4)
out4 = max_pool_2(out4)

concatenate = tf.stack([out1, out2, out3, out4])
mean_pool = tf.reduce_mean(concatenate, axis=0)
flatten = tf.keras.layers.Flatten()(mean_pool)
dense_1 = tf.keras.layers.Dense(64, activation='relu')(flatten)
dense_2 = tf.keras.layers.Dense(64, activation='relu')(dense_1)
outputs = tf.keras.layers.Dense(5, activation='softmax')(dense_2)

model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name='funcional')
optimizer = tf.keras.optimizers.Adam()

In [ ]:
model.summary()

### Cross Entropy

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer, loss, metrics=['accuracy'])
nombre_ensayo = './logs/ce26_4'
summary = model.fit(x=x_train, y=y_train, batch_size=1460, epochs=22, verbose=1, validation_data=(x_val, y_val), shuffle=True, callbacks=[tf.keras.callbacks.TensorBoard(nombre_ensayo, update_freq=100)])

In [ ]:
#Testing
model.evaluate(x=x_test, y=y_test, batch_size=100, verbose=1)

### Entropia Regulada

#### Construcción Función

In [ ]:
"""Implement Regulated Cross Entropy"""

import tensorflow as tf
import tensorflow.keras.backend as K
#import tensorflow_addons as tfa

from tensorflow.python.keras.losses import LossFunctionWrapper
#from tensorflow_addons.utils.keras_utils import LossFunctionWrapper
from tensorflow_addons.utils.types import FloatTensorLike, TensorLike
from typeguard import typechecked
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops

class CrossEntropyRegulated(LossFunctionWrapper):
    @typechecked
    def __init__(
        self,
        from_logits: bool = False,
        beta: FloatTensorLike = 0.5,
        name: str = "crossentropy_regulated",
    ):
        super().__init__(
            crossentropy_regulated,
            name=name,
            from_logits=from_logits,
            beta=beta)

@tf.function
def crossentropy_regulated(
    y_true: TensorLike,
    y_pred: TensorLike,
    beta: FloatTensorLike = 0.5,
    from_logits: bool = False,
) -> tf.Tensor:
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.convert_to_tensor(y_true, dtype=y_pred.dtype)

    '''
    print('y_pred: '+str(y_pred.shape))
    print(y_pred[0][0])
    print('y_pred: '+str(y_true.shape))
    print(y_true[0][0])
    '''

    # Get the cross_entropy for each entry
    ce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=from_logits)
    
    
    #se cambia formaro para que calzen las dimensiones
    #y_pred2=ops.convert_to_tensor_v2_with_dispatch(y_pred)
    y_pred2 = math_ops.cast(y_true, y_true.dtype)

    #ce2 = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=from_logits)
    ce2=K.sparse_categorical_crossentropy(y_pred2, y_pred, from_logits=from_logits)
    

    # compute the final loss and return
    return tf.reduce_mean(ce)+beta*tf.reduce_mean(ce2)

In [ ]:
#Para esta funcion de costos se necesita pasar a float. 
y_val=y_val.astype(np.float32)
y_train=y_train.astype(np.float32)

In [ ]:
#se compila con la nueva funcion de costos
#f1 = tfa.losses.SigmoidFocalCrossEntropy()
f1=CrossEntropyRegulated()

#### Compilación y entrenamiento

In [ ]:
model.compile(optimizer, loss=f1, metrics=['accuracy'])
nombre_ensayo = './logs/er26_2'
summary = model.fit(x=x_train, y=y_train, batch_size=1460, epochs=26, verbose=1, validation_data=(x_val, y_val), shuffle=True, callbacks=[tf.keras.callbacks.TensorBoard(nombre_ensayo, update_freq=100)])

In [ ]:
model.evaluate(x=x_test.astype(np.float32), y=y_test.astype(np.float32), batch_size=100, verbose=1)

## Métricas y Matriz de Confusión

In [ ]:
#Testing
model.evaluate(x=x_test.astype(np.float32), y=y_test.astype(np.float32), batch_size=100, verbose=1)

In [ ]:
#Matriz de confusión
from sklearn.metrics import confusion_matrix
import seaborn as sn

y_test=y_test.astype(np.float32)
x_test=x_test.astype(np.float32)

prediction = model.predict(x_test, batch_size=32, verbose=1)  
y_predicted = np.argmax(prediction, axis=1)  

cnf_matrix = confusion_matrix(y_test, y_predicted)/200
scnn_df_cm = pd.DataFrame(cnf_matrix, range(5), range(5))
plt.figure(figsize = (10,8))  
sn.set(font_scale=1.4) #for label size  
sn.heatmap(scnn_df_cm, annot=True, cmap="Blues", annot_kws={"size": 12}) # font size 
plt.title('Matriz de confusión normalizada') 
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

In [ ]:
FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)

accuracy = 100.0 * (TP + TN) / (TP + TN + FP + FN)
precision = 100.0 * TP / (TP + FP)
recall = 100.0 * TP / (TP + FN)

print('accuracy :'+str(accuracy[1]))
print('precision: '+str(precision[1]))
print('recall: '+str(recall[1]))

## Tensorboard

In [ ]:
%reload_ext tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=.